In [ ]:
#export
from local.torch_basics import *
from local.test import *
from local.core import *
from local.data.all import *
from local.notebook.showdoc import show_doc

In [ ]:
#export
pd.set_option('mode.chained_assignment','raise')

In [ ]:
#default_exp tabular.core

# Tabular core

> Basic function to preprocess tabular data before assembling it in a `DataBunch`.

## Tabular -

In [ ]:
#export
class Tabular(CollBase):
    "A `DataFrame` wrapper that knows which cols are cont/cat/y, and returns rows in `__getitem__`"
    def __init__(self, df, cat_names=None, cont_names=None, y_names=None, is_y_cat=True, split=None):
        super().__init__(df)
        store_attr(self, 'y_names,is_y_cat,split')
        self.cat_names,self.cont_names = L(cat_names),L(cont_names)
        self.cat_y  = None if not is_y_cat else y_names
        self.cont_y = None if     is_y_cat else y_names

    def _new(self, df):
        return Tabular(df, self.cat_names, self.cont_names, self.y_names, is_y_cat=self.is_y_cat, split=self.split)

    def set_col(self,k,v): super().__setitem__(k, v)
    def show(self, max_n=10, **kwargs): display_df(self.all_cols[:max_n])

    def __getitem__(self, idxs):
        "Get/set rows by iloc and cols by name"
        rows,cols = idxs if isinstance(idxs,tuple) else (idxs,None)
        f = self.items.columns.get_loc
        c = (slice(None) if cols is None
             else L(cols).mapped(f) if is_listy(cols)
             else f(cols))
        return self._new(self.items.iloc[rows, c])

    def __getattr__(self,k):
        if k.startswith('_') or k=='items': raise AttributeError
        return getattr(self.items,k)

    @property
    def iloc(self): return self
    @property
    def targ(self): return self.items[self.y_names]
    @property
    def all_cont_names(self): return self.cont_names + self.cont_y
    @property
    def all_cat_names (self): return self.cat_names  + self.cat_y
    @property
    def all_col_names (self): return self.all_cont_names + self.all_cat_names

In [ ]:
#export
class TabularPandas(Tabular):
    def transform(self, cols, f): self.set_col(cols, self[:,cols].transform(f))

In [ ]:
#export
def _add_prop(cls, nm):
    prop = property(lambda o: o.items[list(getattr(o,nm+'_names'))])
    setattr(cls, nm+'s', prop)
    def _f(o,v): o.set_col(getattr(o,nm+'_names'), v)
    setattr(cls, nm+'s', prop.setter(_f))

_add_prop(Tabular, 'cat')
_add_prop(Tabular, 'all_cat')
_add_prop(Tabular, 'cont')
_add_prop(Tabular, 'all_cont')
_add_prop(Tabular, 'all_col')

In [ ]:
df = pd.DataFrame({'a':[0,1,2,0,2], 'b':[0,0,0,0,1]})
to = TabularPandas(df, 'a')
t = pickle.loads(pickle.dumps(to))
test_eq(t.items,to.items)
to.show() # only shows 'a' since that's the only col in `TabularPandas`

In [ ]:
#export
class TabularProc(InplaceTransform):
    "Base class to write a non-lazy tabular processor for dataframes"
    def setup(self, items=None):
        super().setup(items)
        # Procs are called as soon as data is available
        return self(items)

In [ ]:
#export
class Categorify(TabularProc, CollBase):
    "Transform the categorical variables to that type."
    order = 1
    def __init__(self): self.items=[]
    def setups(self, to):
        to.classes = self.items = {n:CategoryMap(to[:to.split,n], add_na=True)
                                   for n in to.all_cat_names}

    def _apply_cats (self, c): return c.cat.codes+1 if is_categorical_dtype(c) else c.map(self[c.name].o2i)
    def _decode_cats(self, c): return c.map(dict(enumerate(self[c.name].items)))
    def encodes(self, to): to.transform(to.all_cat_names, self._apply_cats)
    def decodes(self, to): to.transform(to.all_cat_names, self._decode_cats)

In [ ]:
show_doc(Categorify, title_level=3)

<h3 id="Categorify" class="doc_header"><code>class</code> <code>Categorify</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>Categorify</code>() :: [`TabularProc`](/tabular.core.html#TabularProc)

Transform the categorical variables to that type.

In [ ]:
cat = Categorify()
df = pd.DataFrame({'a':[0,1,2,0,2]})
to = TabularPandas(df, 'a')
cat.setup(to)
test_eq(cat['a'], ['#na#',0,1,2])
test_eq(df['a'], [1,2,3,1,3])
df1 = pd.DataFrame({'a':[1,0,3,-1,2]})

to1 = TabularPandas(df1, 'a')
cat(to1)
#Values that weren't in the training df are sent to 0 (na)
test_eq(df1['a'], [2,1,0,0,3])
to2 = cat.decode(to1)
test_eq(to2.a, [1,0,'#na#','#na#',2])

In [ ]:
#test with splits
cat = Categorify()
df = pd.DataFrame({'a':[0,1,2,3,2]})
to = TabularPandas(df, 'a', split=3)
cat.setup(to)
test_eq(cat['a'], ['#na#',0,1,2])
test_eq(df['a'], [1,2,3,0,3])

In [ ]:
df = pd.DataFrame({'a':pd.Categorical(['M','H','L','M'], categories=['H','M','L'], ordered=True)})
to = TabularPandas(df, 'a')
cat = Categorify()
cat.setup(to)
test_eq(cat['a'], ['#na#','H','M','L'])
test_eq(df.a, [2,1,3,2])
to2 = cat.decode(to)
test_eq(to2.a, ['M','H','L','M'])

In [ ]:
#export
class Normalize(TabularProc):
    "Normalize the continuous variables."
    order = 2
    def setups(self, to):
        df = to[:to.split, to.cont_names]
        self.means,self.stds = df.mean(),df.std(ddof=0)+1e-7

    def encodes(self, to): to.conts = (to.conts-self.means) / self.stds
    def decodes(self, to): to.conts = (to.conts*self.stds ) + self.means

In [ ]:
show_doc(Normalize, title_level=3)

<h3 id="Normalize" class="doc_header"><code>class</code> <code>Normalize</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>Normalize</code>(**`enc`**=*`None`*, **`dec`**=*`None`*, **`filt`**=*`None`*, **`as_item`**=*`False`*) :: [`TabularProc`](/tabular.core.html#TabularProc)

Normalize the continuous variables.

In [ ]:
norm = Normalize()
df = pd.DataFrame({'a':[0,1,2,3,4]})
to = TabularPandas(df, cont_names='a')
norm.setup(to)
x = np.array([0,1,2,3,4])
m,s = x.mean(),x.std()
test_eq(norm.means['a'], m)
test_close(norm.stds['a'], s)
test_close(df['a'].values, (x-m)/s)
df1 = pd.DataFrame({'a':[5,6,7]})
to1 = TabularPandas(df1, cont_names='a')
norm(to1)
test_close(df1['a'].values, (np.array([5,6,7])-m)/s)
to2 = norm.decode(to1)
test_close(to2.a.values, [5,6,7])

In [ ]:
norm = Normalize()
df = pd.DataFrame({'a':[0,1,2,3,4]})
to = TabularPandas(df, cont_names='a', split=3)
norm.setup(to)
x = np.array([0,1,2])
m,s = x.mean(),x.std()
test_eq(norm.means['a'], m)
test_close(norm.stds['a'], s)
test_close(df['a'].values, (np.array([0,1,2,3,4])-m)/s)

In [ ]:
#export
class FillStrategy:
    "Namespace containing the various filling strategies."
    def median  (c,fill): return c.median()
    def constant(c,fill): return fill
    def mode    (c,fill): return c.dropna().value_counts().idxmax()

In [ ]:
#export
class FillMissing(TabularProc):
    "Fill the missing values in continuous columns."
    def __init__(self, fill_strategy=FillStrategy.median, add_col=True, fill_vals=None):
        if fill_vals is None: fill_vals = defaultdict(int)
        store_attr(self, 'fill_strategy,add_col,fill_vals')

    def setups(self, to):
        df = to[:to.split, to.cont_names].items
        self.na_dict = {n:self.fill_strategy(df[n], self.fill_vals[n])
                        for n in pd.isnull(to.conts).any().keys()}

    def encodes(self, to):
        missing = pd.isnull(to.conts)
        for n in missing.any().keys():
            assert n in self.na_dict, f"nan values in `{n}` but not in setup training set"
            to[:,n].fillna(self.na_dict[n], inplace=True)
            if self.add_col:
                to.loc[:,n+'_na'] = missing[n]
                if n+'_na' not in to.cat_names: to.cat_names.append(n+'_na')

In [ ]:
show_doc(FillMissing, title_level=3)

<h3 id="FillMissing" class="doc_header"><code>class</code> <code>FillMissing</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>FillMissing</code>(**`fill_strategy`**=*`'median'`*, **`add_col`**=*`True`*, **`fill_vals`**=*`None`*) :: [`TabularProc`](/tabular.core.html#TabularProc)

Fill the missing values in continuous columns.

In [ ]:
fill1,fill2,fill3 = (FillMissing(fill_strategy=s) 
                     for s in [FillStrategy.median, FillStrategy.constant, FillStrategy.mode])
df = pd.DataFrame({'a':[0,1,np.nan,1,2,3,4]})
df1 = df.copy(); df2 = df.copy()
to,to1,to2 = TabularPandas(df, cont_names='a'),TabularPandas(df1, cont_names='a'),TabularPandas(df2, cont_names='a')
fill1.setup(to); fill2.setup(to1); fill3.setup(to2)
test_eq(fill1.na_dict, {'a': 1.5})
test_eq(fill2.na_dict, {'a': 0})
test_eq(fill3.na_dict, {'a': 1.0})

for t in [to, to1, to2]: test_eq(t.cat_names, ['a_na'])

for to_,v in zip([to, to1, to2], [1.5, 0., 1.]):
    test_eq(to_.a.values, np.array([0, 1, v, 1, 2, 3, 4]))
    test_eq(to_.a_na.values, np.array([0, 0, 1, 0, 0, 0, 0]))
    
dfa = pd.DataFrame({'a':[np.nan,0,np.nan]})
dfa1 = dfa.copy(); dfa2 = dfa.copy()
to,to1,to2 = TabularPandas(dfa, cont_names='a'),TabularPandas(dfa1, cont_names='a'),TabularPandas(dfa2, cont_names='a')
fill1(to); fill2(to1); fill3(to2)
for to_,v in zip([to, to1, to2], [1.5, 0., 1.]):
    test_eq(to_.a.values, np.array([v, 0, v]))
    test_eq(to_.a_na.values, np.array([1, 0, 1]))

## TabularPandas Pipelines -

In [ ]:
procs = [Normalize(), Categorify(), FillMissing(), noop]
proc = Pipeline(procs)

#Test reordering and partialize
test_eq(L(proc.fs).mapped(type), [FillMissing, Transform, Categorify, Normalize])

df = pd.DataFrame({'a':[0,1,2,1,1,2,0], 'b':[0,1,np.nan,1,2,3,4]})
to = TabularPandas(df, 'a', 'b')

#Test setup and apply on df_trn
proc.setup(to)
test_eq(to.cat_names, ['a', 'b_na'])
test_eq(to.a, [1,2,3,2,2,3,1])
test_eq(to.b_na, [1,1,2,1,1,1,1])
x = np.array([0,1,1.5,1,2,3,4])
m,s = x.mean(),x.std()
test_close(to.b.values, (x-m)/s)
test_eq(proc.classes, {'a': ['#na#',0,1,2], 'b_na': ['#na#',False,True]})

In [ ]:
#Test apply on y_names
procs = [Normalize(), Categorify(), FillMissing(), noop]
proc = Pipeline(procs)

df = pd.DataFrame({'a':[0,1,2,1,1,2,0], 'b':[0,1,np.nan,1,2,3,4], 'c': ['b','a','b','a','a','b','a']})
to = TabularPandas(df, 'a', 'b', y_names='c')
proc.setup(to)
test_eq(to.cat_names, ['a', 'b_na'])
test_eq(to.a, [1,2,3,2,2,3,1])
test_eq(to.b_na, [1,1,2,1,1,1,1])
test_eq(to.c, [2,1,2,1,1,2,1])
x = np.array([0,1,1.5,1,2,3,4])
m,s = x.mean(),x.std()
test_close(to.b.values, (x-m)/s)
test_eq(proc.classes, {'a': ['#na#',0,1,2], 'b_na': ['#na#',False,True], 'c': ['#na#','a','b']})

In [ ]:
#export
@delegates(TabularPandas)
def process_df(df, procs, inplace=True, splits=None, **kwargs):
    "Process `df` with `procs` and returns the processed dataframe and the `TabularProcessor` associated"
    df = df if inplace else df.copy()
    if splits is not None: df = df.iloc[sum(splits, [])].reset_index(drop=True)
    split = None if splits is None else len(splits[0])
    to = TabularPandas(df, split=split, **kwargs)
    proc = Pipeline(procs)
    proc.setup(to)
    return to,proc

In [ ]:
procs = [Normalize, Categorify, FillMissing, noop]
df = pd.DataFrame({'a':[0,1,2,1,1,2,0], 'b':[0,1,np.nan,1,2,3,4], 'c': ['b','a','b','a','a','b','a']})
to,proc = process_df(df, procs, cat_names='a', cont_names='b', y_names='c', inplace=False)
test_eq(to.cat_names, ['a', 'b_na'])
test_eq(to.a, [1,2,3,2,2,3,1])
test_eq(df.a.dtype,int)
test_eq(to.b_na, [1,1,2,1,1,1,1])
test_eq(to.c, [2,1,2,1,1,2,1])

In [ ]:
procs = [Normalize, Categorify, FillMissing, noop]
df = pd.DataFrame({'a':[0,1,2,1,1,2,0], 'b':[0,np.nan,1,1,2,3,4], 'c': ['b','a','b','a','a','b','a']})
to,proc = process_df(df, procs, cat_names='a', cont_names='b', y_names='c', inplace=True, splits=[[0,1,4,6], [2,3,5]])
test_eq(to.cat_names, ['a', 'b_na'])
test_eq(to.a, [1,2,2,1,0,2,0])
test_eq(df.a.dtype,int)
test_eq(to.b_na, [1,2,1,1,1,1,1])
test_eq(to.c, [2,1,1,1,2,1,2])

Pass the same `splits` as you will use for splitting the data, so that the setup is only done on the training set. `cat_names` are the names of the categorical variables, `cont_names` the continous ones, `y_names` are the names of the dependent variables that are categories. If `inplace=True`, processing is applied inplace, otherwis it creates a copy of `df`.

In [ ]:
#export
class ReadTabBatch(ItemTransform):
    def __init__(self, proc): self.proc = proc
    def encodes(self, to): return (tensor(to.cats).long(),tensor(to.conts).float()), tensor(to.targ).long()

    def decodes(self, o):
        (cats,conts),targs = to_np(o)
        vals = np.concatenate([cats,conts,targs[:,None]], axis=1)
        df = pd.DataFrame(vals, columns=self.proc.cat_names+self.proc.cont_names+self.proc.y_names)
        to = TabularPandas(df, self.proc.cat_names, self.proc.cont_names, self.proc.y_names, is_y_cat=self.proc.cat_y is not None)
        to = self.proc.decode(to)
        return to

In [ ]:
#export
@delegates()
class TabDataLoader(TfmdDL):
    do_item = noops
    def __init__(self, dataset, proc, bs=16, shuffle=False, after_batch=None, num_workers=0, **kwargs):
        after_batch = L(after_batch)+ReadTabBatch(proc)
        super().__init__(dataset, bs=bs, shuffle=shuffle, after_batch=after_batch, num_workers=num_workers, **kwargs)

    def create_batch(self, b): return self.dataset.items[b]

## Integration example

In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')
df_trn,df_tst = df.iloc[:10000].copy(),df.iloc[10000:].copy()
df_trn.head()

age          workclass  fnlwgt     education  education-num  \
0   49            Private  101320    Assoc-acdm           12.0   
1   44            Private  236746       Masters           14.0   
2   38            Private   96185       HS-grad            NaN   
3   38       Self-emp-inc  112847   Prof-school           15.0   
4   42   Self-emp-not-inc   82297       7th-8th            NaN   

        marital-status        occupation    relationship                 race  \
0   Married-civ-spouse               NaN            Wife                White   
1             Divorced   Exec-managerial   Not-in-family                White   
2             Divorced               NaN       Unmarried                Black   
3   Married-civ-spouse    Prof-specialty         Husband   Asian-Pac-Islander   
4   Married-civ-spouse     Other-service            Wife                Black   

       sex  capital-gain  capital-loss  hours-per-week  native-country salary  
0   Female             0          1902              40   United-States  >=50k  
1     Male         10520             0              45   United-States  >=50k  
2   Female             0             0              32   United-States   <50k  
3     Male             0             0              40   United-States  >=50k  
4   Female             0             0              50   United-States   <50k

In [ ]:
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
procs = [Categorify(), FillMissing(), Normalize()]

splits = RandomSplitter()(range_of(df_trn))

In [ ]:
%time to,proc = process_df(df_trn, procs, splits=splits, cat_names=cat_names, cont_names=cont_names, y_names="salary")

CPU times: user 429 ms, sys: 0 ns, total: 429 ms
Wall time: 429 ms


In [ ]:
filts = [list(range(len(splits[0]))), list(range(len(splits[0]), 10000))]
dsrc = DataSource(to, filts=filts, tfms=[None])
dl = TabDataLoader(dsrc.valid, proc, bs=64, num_workers=0)
dl.show_batch()

In [ ]:
to_tst = TabularPandas(df_tst, cat_names, cont_names, y_names="salary")
proc(to_tst)
to_tst.all_cols.head()

age    fnlwgt  education-num  workclass  education  \
10000  0.461170  1.335778       1.179782          5         10   
10001 -0.931596  1.249125      -0.422230          5         12   
10002  1.047598  0.151123      -1.223237          5          2   
10003  0.534474 -0.280144      -0.422230          5         12   
10004  0.754384  1.437878       0.378776          6          9   

       marital-status  occupation  relationship  race  age_na  fnlwgt_na  \
10000               3           2             1     2       1          1   
10001               3          15             1     4       1          1   
10002               1           9             2     5       1          1   
10003               7           2             5     5       1          1   
10004               3           5             1     5       1          1   

       education-num_na  salary  
10000                 1       1  
10001                 1       1  
10002                 1       1  
10003                 1       1  
10004                 1       2

## Not being used now - for multi-modal

In [ ]:
class TensorTabular(TupleBase):
    def get_ctxs(self, max_n=10, **kwargs):
        n_samples = min(self[0].shape[0], max_n)
        df = pd.DataFrame(index = range(n_samples))
        return [df.iloc[i] for i in range(n_samples)]

    def display(self, ctxs): display_df(pd.DataFrame(ctxs))

class TabularLine(pd.Series):
    "A line of a dataframe that knows how to show itself"
    def show(self, ctx=None, **kwargs): return self if ctx is None else ctx.append(self)

class ReadTabLine(ItemTransform):
    def __init__(self, proc): self.proc = proc

    def encodes(self, row):
        cats,conts = (o.mapped(row.__getitem__) for o in (self.proc.cat_names,self.proc.cont_names))
        return TensorTabular(tensor(cats).long(),tensor(conts).float())

    def decodes(self, o):
        to = TabularPandas(o, self.proc.cat_names, self.proc.cont_names, self.proc.y_names)
        to = self.proc.decode(to)
        return TabularLine(pd.Series({c: v for v,c in zip(to.items[0]+to.items[1], self.proc.cat_names+self.proc.cont_names)}))

class ReadTabTarget(ItemTransform):
    def __init__(self, proc): self.proc = proc
    def encodes(self, row): return row[self.proc.y_names].astype(np.int64)
    def decodes(self, o): return Category(self.proc.classes[self.proc.y_names][o])

In [ ]:
# tds = TfmdDS(to.items, tfms=[[ReadTabLine(proc)], ReadTabTarget(proc)])
# enc = tds[1]
# test_eq(enc[0][0], tensor([2,1]))
# test_close(enc[0][1], tensor([-0.628828]))
# test_eq(enc[1], 1)

# dec = tds.decode(enc)
# assert isinstance(dec[0], TabularLine)
# test_close(dec[0], pd.Series({'a': 1, 'b_na': False, 'b': 1}))
# test_eq(dec[1], 'a')

# test_stdout(lambda: print(tds.show_at(1)), """a               1
# b_na        False
# b               1
# category        a
# dtype: object""")

## Export -

In [ ]:
#hide
from local.notebook.export import notebook2script
notebook2script(all_fs=True)

Converted 00_test.ipynb.
Converted 01_core.ipynb.
Converted 01a_torch_core.ipynb.
Converted 01b_script.ipynb.
Converted 01c_dataloader.ipynb.
Converted 02_data_transforms.ipynb.
Converted 03_data_pipeline.ipynb.
Converted 05_data_core.ipynb.
Converted 06_data_source.ipynb.
Converted 07_vision_core.ipynb.
Converted 08_pets_tutorial.ipynb.
Converted 09_vision_augment.ipynb.
Converted 11_layers.ipynb.
Converted 11a_vision_models_xresnet.ipynb.
Converted 12_optimizer.ipynb.
Converted 13_learner.ipynb.
Converted 14_callback_schedule.ipynb.
Converted 15_callback_hook.ipynb.
Converted 16_callback_progress.ipynb.
Converted 17_callback_tracker.ipynb.
Converted 18_callback_fp16.ipynb.
Converted 19_callback_mixup.ipynb.
Converted 20_metrics.ipynb.
Converted 21_tutorial_imagenette.ipynb.
Converted 22_vision_learner.ipynb.
Converted 23_tutorial_transfer_learning.ipynb.
Converted 30_text_core.ipynb.
Converted 31_text_data.ipynb.
Converted 32_text_models_awdlstm.ipynb.
Converted 33_text_models_core.i